In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pandas as pd
import numpy as np


from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
raid_log = 'RAID_SHIELD_DB.INGESTION.RAID_LOG__C'
use_case = 'RAID_SHIELD_DB.INGESTION.USE_CASE__C'
account_name = 'LLM_PM_CORTEX.PUBLIC.ACCOUNTID2ACCOUNTNAME'

In [ ]:
account_name = session.sql(f"select * from {account_name}")
account_name.show()

In [ ]:
df_raid_log = session.sql(f"select * from {raid_log}")
df_raid_log.show()

In [ ]:
df_use_case = session.sql(f"select * from {use_case}")
df_use_case.show()

In [ ]:
df_raid_log = df_raid_log.to_pandas()
df_use_case = df_use_case.to_pandas()

In [ ]:
df_raid_log.columns

In [ ]:
df_raid_log.rename(columns={'ID': 'RAID_ID', 'NAME': 'RAID_NAME'}, inplace=True)

In [ ]:
df_use_case.rename(columns={'ID': 'USECASE_ID', 'NAME': 'USECASE_NAME'}, inplace=True)

In [ ]:
df_use_case = df_use_case[['PROJECT__C','SOLUTION_ARCHITECTURE__C','INDUSTRY__C','FLOW_OF_EVENTS__C','DESCRIPTION__C','USE_CASE_TYPE__C','MARKET_SE__C','SOURCE__C','USECASE_ID','USECASE_NAME']]

In [ ]:
df_raid_log = df_raid_log[['RAID_ID','RAID_NAME','OWNER__C','STAGE__C','STATUS__C','ACCOUNT__C','PROJECT__C','PRIORITY__C','USE_CASE__C','KIPI_OWNER__C','RAID_CATEGORY__C','CLOSURE_REASON__C','RAID_MITIGATION__C','POTENTIAL_IMPACT__C','RAID_DESCRIPTION__C','MITIGATION_ACTION__C','RAID_OTHER_REMARKS__C']]

In [ ]:
df_raid_log.columns

In [ ]:
df_raid_log = df_raid_log.dropna(subset=['PROJECT__C'])
df_raid_log = df_raid_log.fillna('Not Available')

In [ ]:
df_use_case = df_use_case.dropna(subset=['PROJECT__C'])
df_use_case = df_use_case.fillna('Not Available')

In [ ]:
len(df_use_case[df_use_case['PROJECT__C']=='Not Available'])

In [ ]:
len(df_raid_log[df_raid_log['PROJECT__C']=='Not Available'])

In [ ]:
spark_raid = session.create_dataframe(df_raid_log)
spark_raid.write.save_as_table('RAID_SHIELD_DB.INGESTION.RAID_LOG__Z', mode='overwrite')  ## save dataframe in snowflake database    'RAID_SHIELD_DB.INGESTION.MERGEDV1' 

In [ ]:
spark_use = session.create_dataframe(df_use_case)
spark_use.write.save_as_table('RAID_SHIELD_DB.INGESTION.USECASE__Z', mode='overwrite')  ## save dataframe in snowflake database    'RAID_SHIELD_DB.INGESTION.MERGEDV1' 

In [ ]:
len(df_use_case)

In [ ]:
len(df_raid_log)

In [ ]:
def merge(df1,df2,uniq_col1,uniq_col2):
    df = df1.merge(df2, 'left', left_on=uniq_col1, right_on=uniq_col2)
    
    # Return value will appear in the Results tab.
    return df

In [ ]:
df_merged = merge(df_raid_log,df_use_case,'PROJECT__C','PROJECT__C')

In [ ]:
account_name = account_name.to_pandas()
account_name.columns

In [ ]:
df_merged = merge(df_merged,account_name,'ACCOUNT__C','ACCOUNT_ID')

In [ ]:
df_merged.head(10)

In [ ]:
df_merged = df_merged.drop(['ACCOUNT_ID'], axis=1)

In [ ]:
df_merged.columns[26]

In [ ]:
colmn = {}

for i in list(df_merged.columns):
    
    colmn[i] = i.replace("__C","")

In [ ]:
colmn

In [ ]:
df_merged.rename(columns=colmn, inplace=True)

In [ ]:
df_merged.columns

In [ ]:
spark_df = session.create_dataframe(df_merged)
spark_df.write.save_as_table('RAID_SHIELD_DB.INGESTION.MERGEDV1', mode='overwrite')  ## save dataframe in snowflake database    'RAID_SHIELD_DB.INGESTION.MERGEDV1' 

In [ ]:
ALTER TABLE RAID_SHIELD_DB.INGESTION.MERGEDV1 ADD COLUMN json_object STRING;

In [ ]:
-- Step 2: Update the new column with the JSON object
UPDATE RAID_SHIELD_DB.INGESTION.MERGEDV1
SET json_object = OBJECT_CONSTRUCT(
    'RAID_ID', RAID_ID,
    'RAID_NAME', RAID_NAME,
    'OWNER', OWNER,
    'STAGE', STAGE,
    'STATUS', STATUS,
    'ACCOUNT', ACCOUNT,
    'PROJECT', PROJECT,
    'PRIORITY', PRIORITY,
    'USE_CASE', USE_CASE,
    'KIPI_OWNER', KIPI_OWNER,
    'RAID_CATEGORY', RAID_CATEGORY,
    'CLOSURE_REASON', CLOSURE_REASON,
    'RAID_MITIGATION',RAID_MITIGATION,
    'POTENTIAL_IMPACT', POTENTIAL_IMPACT,
    'RAID_DESCRIPTION', RAID_DESCRIPTION,
    'MITIGATION_ACTION', MITIGATION_ACTION,
    'RAID_OTHER_REMARKS', RAID_OTHER_REMARKS,
    'SOLUTION_ARCHITECTURE', SOLUTION_ARCHITECTURE,
    'INDUSTRY', INDUSTRY,
    'FLOW_OF_EVENTS', FLOW_OF_EVENTS,
    'DESCRIPTION' , DESCRIPTION,
    'USE_CASE_TYPE',USE_CASE_TYPE,
    'MARKET_SE',MARKET_SE,
    'SOURCE', SOURCE,
    'USECASE_ID' , USECASE_ID,
    'USECASE_NAME',USECASE_NAME,
    'ACCOUNT_NAME',ACCOUNT_NAME
    
  )::STRING;

In [ ]:
SELECT 
  OBJECT_CONSTRUCT(
    'RAID_ID', RAID_ID,
    'RAID_NAME', RAID_NAME,
    'OWNER', OWNER,
    'STAGE', STAGE,
    'STATUS', STATUS,
    'ACCOUNT', ACCOUNT,
    'PROJECT', PROJECT,
    'PRIORITY', PRIORITY,
    'USE_CASE', USE_CASE,
    'KIPI_OWNER', KIPI_OWNER,
    'RAID_CATEGORY', RAID_CATEGORY,
    'CLOSURE_REASON', CLOSURE_REASON,
    'RAID_MITIGATION',RAID_MITIGATION,
    'POTENTIAL_IMPACT', POTENTIAL_IMPACT,
    'RAID_DESCRIPTION', RAID_DESCRIPTION,
    'MITIGATION_ACTION', MITIGATION_ACTION,
    'RAID_OTHER_REMARKS', RAID_OTHER_REMARKS,
    'SOLUTION_ARCHITECTURE', SOLUTION_ARCHITECTURE,
    'INDUSTRY', INDUSTRY,
    'FLOW_OF_EVENTS', FLOW_OF_EVENTS,
    'DESCRIPTION' , DESCRIPTION,
    'USE_CASE_TYPE',USE_CASE_TYPE,
    'MARKET_SE',MARKET_SE,
    'SOURCE', SOURCE,
    'USECASE_ID' , USECASE_ID,
    'USECASE_NAME',USECASE_NAME,
    'ACCOUNT_NAME',ACCOUNT_NAME
  ) AS json_object
FROM RAID_SHIELD_DB.INGESTION.MERGEDV1;

In [ ]:
ALTER TABLE RAID_SHIELD_DB.INGESTION.MERGEDV1 ADD COLUMN embedding VECTOR(FLOAT, 768);

In [ ]:
UPDATE RAID_SHIELD_DB.INGESTION.MERGEDV1 SET embedding = snowflake.cortex.embed_text('e5-base-v2', json_object);

In [ ]:
SELECT * FROM RAID_SHIELD_DB.INGESTION.MERGEDV1;

In [ ]:
SELECT JSON_OBJECT FROM RAID_SHIELD_DB.INGESTION.MERGEDV1;

## Shivangi work starts here 


In [ ]:
df = session.sql(f"select * from RAID_SHIELD_DB.INGESTION.MERGEDV1")
df.show()

In [ ]:
df= df.drop_duplicates()

In [ ]:
#modified_query ='Give me possible raid for migration from SQL server to snowflake'
modified_query ='Our team is integrating Snowflake with an ETL tool. What RAIDs could arise during the integration'

In [ ]:
context = session.sql(f"""
                            (SELECT LISTAGG(JSON_OBJECT , ', ') WITHIN GROUP (ORDER BY JSON_OBJECT) AS context
                                from
                            (  SELECT JSON_OBJECT  FROM RAID_SHIELD_DB.INGESTION.MERGEDV1 
                                ORDER BY vector_l2_distance(
                                snowflake.cortex.embed_text('e5-base-v2', 
                                '{modified_query}'
                                ), EMBEDDING
                                ) LIMIT 30))
                            """).collect()[0][0]
   
    
print(context)

In [ ]:
import json
context_data = json.loads(f"[{context}]")  # Convert the aggregated JSON string to a Python list of dictionaries

# Step 2: Convert to Pandas DataFrame
context_df = pd.DataFrame(context_data)

In [ ]:
context_df.head(30)

In [ ]:
# Identify the first occurrence of each category
desired_order = ['Risk', 'Assumption', 'Issue', 'Dependency']

# Create a categorical column with the desired order
context_df['RAID_CATEGORY'] = pd.Categorical(context_df['RAID_CATEGORY'], categories=desired_order, ordered=True)

# Sort the DataFrame by the categorical order
sorted_df = context_df.sort_values(by='RAID_CATEGORY')
first_occurrences = context_df.groupby('RAID_CATEGORY', as_index=False).first()

# Display the resulting DataFrame
first_occurrences.head()

In [ ]:
session.write_pandas(first_occurrences, table_name="RAID_SHIELD_DB.INGESTION.SIMILARITY_MATCH_RESULTS", overwrite=True)